In [1]:
import pandas as pd

In [10]:
file_path = 'data/well_data.xlsx'
df = pd.read_excel(file_path)
df

,date,pressure_fact,pressure_smoothed
0,2005-01-11,86.8000,87.119052
1,2005-01-12,NaN,87.165231
2,2005-01-19,NaN,86.788705
3,2005-01-20,86.8000,86.648587
4,2005-01-21,NaN,86.490297
...,...,...,...
3048,2025-03-11,74.2344,74.448930
3049,2025-03-12,NaN,74.386106
3050,2025-03-13,NaN,74.330345
3051,2025-03-15,74.2344,74.240182


In [31]:
import numpy as np
from datetime import datetime, timedelta

def find_extremes_improved(df, min_distance_days=90, prominence_percent=3, max_cycle_days=400):
    """
    Улучшенный алгоритм поиска экстремумов для циклических данных с годовыми циклами.
    """
    
    # Копируем датафрейм
    result_df = df.copy()
    
    # Преобразуем дату
    if not pd.api.types.is_datetime64_any_dtype(result_df['date']):
        result_df['date'] = pd.to_datetime(result_df['date'], format='%d.%m.%Y')
    
    # Сортируем по дате
    result_df = result_df.sort_values('date').reset_index(drop=True)
    
    # Инициализируем колонки
    result_df['maxima'] = np.nan
    result_df['minima'] = np.nan
    
    # Получаем значения
    pressures = result_df['pressure_smoothed'].values
    dates = result_df['date'].values
    n = len(pressures)
    
    # Вычисляем среднее значение для определения значимости
    avg_pressure = np.nanmean(pressures)
    min_prominence = avg_pressure * (prominence_percent / 100)
    
    # Конвертируем расстояние в днях в индексы
    # Оцениваем среднее расстояние между точками
    if n > 1:
        avg_days_between_points = (dates[-1] - dates[0]).astype('timedelta64[D]').astype(int) / (n - 1)
        min_distance_points = int(min_distance_days / avg_days_between_points)
    else:
        min_distance_points = 30
    
    # Используем scipy.signal для поиска пиков (если установлен)
    try:
        from scipy.signal import find_peaks
        use_scipy = True
    except ImportError:
        use_scipy = False
        print("Библиотека scipy не установлена. Используется упрощенный алгоритм.")
    
    if use_scipy:
        # Находим максимумы
        max_peaks, _ = find_peaks(pressures, 
                                  distance=min_distance_points,
                                  prominence=min_prominence)
        
        # Находим минимумы (инвертируем сигнал)
        min_peaks, _ = find_peaks(-pressures, 
                                  distance=min_distance_points,
                                  prominence=min_prominence)
        
        maxima_indices = max_peaks.tolist()
        minima_indices = min_peaks.tolist()
    else:
        # Упрощенный алгоритм без scipy
        maxima_indices = []
        minima_indices = []
        
        # Ищем локальные максимумы
        for i in range(min_distance_points, n - min_distance_points):
            window = pressures[i-min_distance_points//2:i+min_distance_points//2+1]
            if pressures[i] == np.max(window) and pressures[i] != window[0] and pressures[i] != window[-1]:
                # Проверяем значимость
                left_min = np.min(pressures[max(0, i-min_distance_points):i])
                right_min = np.min(pressures[i:min(n, i+min_distance_points)])
                prominence_val = pressures[i] - max(left_min, right_min)
                
                if prominence_val > min_prominence:
                    maxima_indices.append(i)
        
        # Ищем локальные минимумы
        for i in range(min_distance_points, n - min_distance_points):
            window = pressures[i-min_distance_points//2:i+min_distance_points//2+1]
            if pressures[i] == np.min(window) and pressures[i] != window[0] and pressures[i] != window[-1]:
                # Проверяем значимость
                left_max = np.max(pressures[max(0, i-min_distance_points):i])
                right_max = np.max(pressures[i:min(n, i+min_distance_points)])
                prominence_val = max(left_max, right_max) - pressures[i]
                
                if prominence_val > min_prominence:
                    minima_indices.append(i)
    
    # Фильтруем экстремумы, чтобы они чередовались
    filtered_maxima = []
    filtered_minima = []
    
    # Объединяем и сортируем все экстремумы
    all_extrema = sorted([(idx, 'max', pressures[idx]) for idx in maxima_indices] + 
                         [(idx, 'min', pressures[idx]) for idx in minima_indices])
    
    # Проходим по всем экстремумам и выбираем наиболее значимые
    i = 0
    while i < len(all_extrema):
        current_idx, current_type, current_val = all_extrema[i]
        
        if current_type == 'max':
            # Ищем ближайший минимум после этого максимума
            min_candidates = []
            for j in range(i+1, len(all_extrema)):
                idx2, type2, val2 = all_extrema[j]
                if type2 == 'min':
                    # Проверяем расстояние
                    days_diff = (dates[idx2] - dates[current_idx]).astype('timedelta64[D]').astype(int)
                    if 30 < days_diff < 300:  # Минимум должен быть в пределах 300 дней
                        min_candidates.append((idx2, val2, days_diff))
            
            if min_candidates:
                # Выбираем самый глубокий минимум
                best_min_idx = min(min_candidates, key=lambda x: x[1])[0]
                
                # Проверяем, что разница достаточно большая
                if pressures[current_idx] - pressures[best_min_idx] > min_prominence:
                    filtered_maxima.append(current_idx)
                    filtered_minima.append(best_min_idx)
                    
                    # Пропускаем обработанные точки
                    while i < len(all_extrema) and all_extrema[i][0] != best_min_idx:
                        i += 1
        i += 1
    
    # Если все еще мало экстремумов, используем более простой подход
    if len(filtered_maxima) < 5:
        print(f"Найдено слишком мало экстремумов: {len(filtered_maxima)} максимумов")
        print("Используем альтернативный алгоритм...")
        
        # Альтернативный простой алгоритм
        filtered_maxima = []
        filtered_minima = []
        
        # Ищем максимумы каждые ~180 дней (полгода)
        i = 0
        while i < n:
            # Определяем окно поиска
            window_start = i
            window_end = min(n, i + 180 // int(avg_days_between_points) if avg_days_between_points > 0 else i + 100)
            
            if window_end - window_start > 10:
                # Ищем максимум в окне
                max_idx = window_start + np.argmax(pressures[window_start:window_end])
                max_val = pressures[max_idx]
                
                # Ищем минимум после максимума
                if max_idx + 50 < n:
                    min_window_start = max_idx + 30 // int(avg_days_between_points) if avg_days_between_points > 0 else max_idx + 15
                    min_window_end = min(n, min_window_start + 150 // int(avg_days_between_points) if avg_days_between_points > 0 else min_window_start + 75)
                    
                    if min_window_end - min_window_start > 10:
                        min_idx = min_window_start + np.argmin(pressures[min_window_start:min_window_end])
                        min_val = pressures[min_idx]
                        
                        # Проверяем значимость
                        if max_val - min_val > min_prominence:
                            filtered_maxima.append(max_idx)
                            filtered_minima.append(min_idx)
                            
                            i = min_idx
                            continue
            
            i += 30 // int(avg_days_between_points) if avg_days_between_points > 0 else i + 15
    
    # Заполняем датафрейм
    for idx in filtered_maxima:
        result_df.loc[idx, 'maxima'] = pressures[idx]
    
    for idx in filtered_minima:
        result_df.loc[idx, 'minima'] = pressures[idx]
    
    return result_df

In [46]:
import numpy as np
from datetime import datetime, timedelta
from scipy.signal import find_peaks

def find_extremes_improved_v2(df, min_distance_days=60, prominence_percent=2, 
                              max_cycle_days=400, edge_buffer_days=30):
    """
    Улучшенный алгоритм поиска экстремумов для циклических данных с годовыми циклами.
    
    Параметры:
    ----------
    df : pandas.DataFrame
        Датафрейм с колонками ['date', 'pressure_smoothed']
    min_distance_days : int
        Минимальное расстояние между экстремумами в днях (по умолчанию 60)
    prominence_percent : float
        Минимальная значимость экстремума в процентах от среднего значения (по умолчанию 2%)
    max_cycle_days : int
        Максимальная длина цикла в днях (по умолчанию 400)
    edge_buffer_days : int
        Буфер для обработки краев данных в днях (по умолчанию 30)
    
    Возвращает:
    -----------
    pandas.DataFrame
        Датафрейм с добавленными колонками 'maxima' и 'minima'
    """
    
    # Копируем датафрейм
    result_df = df.copy()
    
    # Преобразуем дату
    if not pd.api.types.is_datetime64_any_dtype(result_df['date']):
        result_df['date'] = pd.to_datetime(result_df['date'], format='%d.%m.%Y')
    
    # Сортируем по дате
    result_df = result_df.sort_values('date').reset_index(drop=True)
    
    # Инициализируем колонки
    result_df['maxima'] = np.nan
    result_df['minima'] = np.nan
    
    # Получаем значения
    pressures = result_df['pressure_smoothed'].values
    dates = result_df['date'].values
    n = len(pressures)
    
    if n < 10:
        print("Слишком мало данных для поиска экстремумов")
        return result_df
    
    # Вычисляем среднее значение для определения значимости
    avg_pressure = np.nanmean(pressures)
    min_prominence = avg_pressure * (prominence_percent / 100)
    
    # Конвертируем расстояние в днях в индексы
    if n > 1:
        avg_days_between_points = (dates[-1] - dates[0]).astype('timedelta64[D]').astype(int) / (n - 1)
        min_distance_points = max(5, int(min_distance_days / avg_days_between_points))
    else:
        min_distance_points = 30
    
    print(f"Всего точек: {n}")
    print(f"Среднее давление: {avg_pressure:.2f}")
    print(f"Минимальная значимость: {min_prominence:.2f}")
    print(f"Минимальное расстояние в точках: {min_distance_points}")
    
    # Инициализируем списки для экстремумов
    all_maxima_indices = []
    all_minima_indices = []
    
    # 1. Поиск с помощью scipy.signal.find_peaks (основной метод)
    try:
        # Находим максимумы
        max_peaks, max_properties = find_peaks(
            pressures, 
            distance=min_distance_points,
            prominence=min_prominence,
            width=min_distance_points//3,  # Учитываем ширину пика
            rel_height=0.5
        )
        
        # Находим минимумы (инвертируем сигнал)
        min_peaks, min_properties = find_peaks(
            -pressures, 
            distance=min_distance_points,
            prominence=min_prominence,
            width=min_distance_points//3,
            rel_height=0.5
        )
        
        all_maxima_indices = list(max_peaks)
        all_minima_indices = list(min_peaks)
        
        print(f"Найдено {len(all_maxima_indices)} максимумов и {len(all_minima_indices)} минимумов через find_peaks")
        
    except Exception as e:
        print(f"Ошибка при использовании find_peaks: {e}")
        all_maxima_indices = []
        all_minima_indices = []
    
    # 2. Дополнительный поиск по годовым циклам
    years = result_df['date'].dt.year.unique()
    yearly_extremes = []
    
    for year in years:
        year_mask = result_df['date'].dt.year == year
        year_indices = np.where(year_mask)[0]
        
        if len(year_indices) > 30:  # Минимум 30 точек в году
            year_pressures = pressures[year_indices]
            
            # Находим максимум года
            year_max_idx_local = np.argmax(year_pressures)
            year_max_idx = year_indices[year_max_idx_local]
            year_max_val = year_pressures[year_max_idx_local]
            
            # Находим минимум года
            year_min_idx_local = np.argmin(year_pressures)
            year_min_idx = year_indices[year_min_idx_local]
            year_min_val = year_pressures[year_min_idx_local]
            
            # Проверяем, что это действительно экстремум в окрестности
            window_size = min(50, len(year_indices)//3)
            
            # Для максимума
            if year_max_idx_local >= window_size and year_max_idx_local < len(year_indices) - window_size:
                window = year_pressures[year_max_idx_local-window_size:year_max_idx_local+window_size+1]
                if year_max_val == np.max(window):
                    yearly_extremes.append((year_max_idx, 'max', year_max_val))
            
            # Для минимума
            if year_min_idx_local >= window_size and year_min_idx_local < len(year_indices) - window_size:
                window = year_pressures[year_min_idx_local-window_size:year_min_idx_local+window_size+1]
                if year_min_val == np.min(window):
                    yearly_extremes.append((year_min_idx, 'min', year_min_val))
    
    print(f"Найдено {len([e for e in yearly_extremes if e[1]=='max'])} максимумов и "
          f"{len([e for e in yearly_extremes if e[1]=='min'])} минимумов по годам")
    
    # 3. Проверка краевых точек
    edge_extremes = []
    
    # Проверяем первые edge_buffer_days дней
    edge_points = int(edge_buffer_days / avg_days_between_points) if avg_days_between_points > 0 else 30
    edge_points = min(edge_points, n//4)
    
    if edge_points > 5:
        # Проверяем начало данных
        start_window = pressures[:edge_points*2]
        if len(start_window) > 0:
            start_max_idx = np.argmax(start_window)
            start_max_val = start_window[start_max_idx]
            start_min_idx = np.argmin(start_window)
            start_min_val = start_window[start_min_idx]
            
            # Проверяем значимость
            if start_max_idx > 0 and start_max_idx < len(start_window)-1:
                if start_max_val - start_min_val > min_prominence:
                    edge_extremes.append((start_max_idx, 'max', start_max_val))
                    edge_extremes.append((start_min_idx, 'min', start_min_val))
        
        # Проверяем конец данных
        end_window = pressures[-edge_points*2:]
        if len(end_window) > 0:
            end_max_idx = n - len(end_window) + np.argmax(end_window)
            end_max_val = pressures[end_max_idx]
            end_min_idx = n - len(end_window) + np.argmin(end_window)
            end_min_val = pressures[end_min_idx]
            
            if end_max_idx > n - len(end_window) and end_max_idx < n-1:
                if end_max_val - end_min_val > min_prominence:
                    edge_extremes.append((end_max_idx, 'max', end_max_val))
                    edge_extremes.append((end_min_idx, 'min', end_min_val))
    
    print(f"Найдено {len([e for e in edge_extremes if e[1]=='max'])} максимумов и "
          f"{len([e for e in edge_extremes if e[1]=='min'])} минимумов на краях")
    
    # 4. Объединяем все найденные экстремумы
    all_extrema_dict = {'max': [], 'min': []}
    
    # Добавляем экстремумы из find_peaks
    for idx in all_maxima_indices:
        all_extrema_dict['max'].append((idx, pressures[idx]))
    for idx in all_minima_indices:
        all_extrema_dict['min'].append((idx, pressures[idx]))
    
    # Добавляем годовые экстремумы
    for idx, typ, val in yearly_extremes:
        all_extrema_dict[typ].append((idx, val))
    
    # Добавляем краевые экстремумы
    for idx, typ, val in edge_extremes:
        all_extrema_dict[typ].append((idx, val))
    
    # Удаляем дубликаты и сортируем
    for typ in ['max', 'min']:
        if all_extrema_dict[typ]:
            # Удаляем дубликаты по индексу
            unique_dict = {}
            for idx, val in all_extrema_dict[typ]:
                if idx not in unique_dict:
                    unique_dict[idx] = val
                elif typ == 'max' and val > unique_dict[idx]:
                    unique_dict[idx] = val
                elif typ == 'min' and val < unique_dict[idx]:
                    unique_dict[idx] = val
            
            # Сортируем по индексу
            all_extrema_dict[typ] = sorted([(idx, val) for idx, val in unique_dict.items()])
    
    print(f"После объединения: {len(all_extrema_dict['max'])} максимумов и {len(all_extrema_dict['min'])} минимумов")
    
    # 5. Фильтрация и чередование экстремумов
    filtered_maxima = []
    filtered_minima = []
    
    # Объединяем все экстремумы в один отсортированный список
    combined_extrema = []
    for idx, val in all_extrema_dict['max']:
        combined_extrema.append((idx, 'max', val))
    for idx, val in all_extrema_dict['min']:
        combined_extrema.append((idx, 'min', val))
    
    combined_extrema.sort(key=lambda x: x[0])
    
    # Алгоритм чередования с допущениями
    i = 0
    last_type = None
    last_idx = -min_distance_points * 2
    
    while i < len(combined_extrema):
        idx, typ, val = combined_extrema[i]
        
        # Проверяем расстояние до предыдущего экстремума
        if idx - last_idx < min_distance_points:
            # Если слишком близко, выбираем более значимый
            if last_type == 'max' and typ == 'max':
                # Два максимума рядом - выбираем больший
                if val > pressures[last_idx]:
                    # Удаляем предыдущий, добавляем текущий
                    if last_idx in filtered_maxima:
                        filtered_maxima.remove(last_idx)
                    filtered_maxima.append(idx)
                    last_idx = idx
                # Иначе пропускаем текущий
            elif last_type == 'min' and typ == 'min':
                # Два минимума рядом - выбираем меньший
                if val < pressures[last_idx]:
                    if last_idx in filtered_minima:
                        filtered_minima.remove(last_idx)
                    filtered_minima.append(idx)
                    last_idx = idx
            i += 1
            continue
        
        # Проверяем чередование
        if last_type is None or typ != last_type:
            # Если это первый экстремум или типы чередуются
            if typ == 'max':
                filtered_maxima.append(idx)
            else:
                filtered_minima.append(idx)
            
            last_type = typ
            last_idx = idx
            i += 1
        else:
            # Если типы не чередуются, проверяем следующий экстремум
            # Ищем ближайший экстремум другого типа в пределах max_cycle_days
            found_alternate = False
            max_search = min(i + 20, len(combined_extrema))
            
            for j in range(i + 1, max_search):
                idx2, typ2, val2 = combined_extrema[j]
                
                # Проверяем расстояние в днях
                days_diff = (dates[idx2] - dates[idx]).astype('timedelta64[D]').astype(int)
                
                if typ2 != typ and 30 < days_diff < max_cycle_days:
                    # Нашли чередующийся экстремум
                    if typ2 == 'max':
                        filtered_maxima.append(idx2)
                    else:
                        filtered_minima.append(idx2)
                    
                    last_type = typ2
                    last_idx = idx2
                    i = j + 1
                    found_alternate = True
                    break
            
            if not found_alternate:
                # Если не нашли чередующийся, пропускаем текущий
                i += 1
    
    # 6. Дополнительная проверка пропущенных экстремумов
    # Ищем крупные пропуски между экстремумами
    all_filtered = sorted([(idx, 'max', pressures[idx]) for idx in filtered_maxima] + 
                         [(idx, 'min', pressures[idx]) for idx in filtered_minima])
    
    for k in range(len(all_filtered) - 1):
        idx1, typ1, val1 = all_filtered[k]
        idx2, typ2, val2 = all_filtered[k + 1]
        
        # Вычисляем расстояние в днях
        days_diff = (dates[idx2] - dates[idx1]).astype('timedelta64[D]').astype(int)
        
        # Если большой пропуск (> 250 дней), ищем экстремум в середине
        if days_diff > 250 and typ1 != typ2:
            mid_idx = (idx1 + idx2) // 2
            search_start = max(0, mid_idx - min_distance_points)
            search_end = min(n, mid_idx + min_distance_points)
            
            if search_end - search_start > 10:
                if typ1 == 'max':
                    # Между максимумом и минимумом должен быть минимум
                    search_min_idx = search_start + np.argmin(pressures[search_start:search_end])
                    search_min_val = pressures[search_min_idx]
                    
                    # Проверяем значимость
                    if val1 - search_min_val > min_prominence and val2 - search_min_val > min_prominence:
                        if search_min_idx not in filtered_minima:
                            filtered_minima.append(search_min_idx)
                else:
                    # Между минимумом и максимумом должен быть максимум
                    search_max_idx = search_start + np.argmax(pressures[search_start:search_end])
                    search_max_val = pressures[search_max_idx]
                    
                    # Проверяем значимость
                    if search_max_val - val1 > min_prominence and search_max_val - val2 > min_prominence:
                        if search_max_idx not in filtered_maxima:
                            filtered_maxima.append(search_max_idx)
    
    # 7. Сортировка и удаление дубликатов
    filtered_maxima = sorted(list(set(filtered_maxima)))
    filtered_minima = sorted(list(set(filtered_minima)))
    
    print(f"После фильтрации: {len(filtered_maxima)} максимумов и {len(filtered_minima)} минимумов")
    
    # 8. Заполняем датафрейм
    for idx in filtered_maxima:
        if 0 <= idx < n:
            result_df.loc[idx, 'maxima'] = pressures[idx]
    
    for idx in filtered_minima:
        if 0 <= idx < n:
            result_df.loc[idx, 'minima'] = pressures[idx]
    
    # 9. Дополнительная информация для отладки
    if len(filtered_maxima) > 0 and len(filtered_minima) > 0:
        print(f"Первый максимум: {result_df.loc[filtered_maxima[0], 'date'].strftime('%d.%m.%Y')} = {pressures[filtered_maxima[0]]:.2f}")
        print(f"Последний максимум: {result_df.loc[filtered_maxima[-1], 'date'].strftime('%d.%m.%Y')} = {pressures[filtered_maxima[-1]]:.2f}")
        print(f"Первый минимум: {result_df.loc[filtered_minima[0], 'date'].strftime('%d.%m.%Y')} = {pressures[filtered_minima[0]]:.2f}")
        print(f"Последний минимум: {result_df.loc[filtered_minima[-1], 'date'].strftime('%d.%m.%Y')} = {pressures[filtered_minima[-1]]:.2f}")
    
    return result_df

In [47]:
df_with_extrems = find_extremes_improved_v2(df)

Всего точек: 3053
Среднее давление: 85.28
Минимальная значимость: 1.71
Минимальное расстояние в точках: 24
Найдено 20 максимумов и 20 минимумов через find_peaks
Найдено 3 максимумов и 12 минимумов по годам
Найдено 1 максимумов и 1 минимумов на краях
После объединения: 21 максимумов и 21 минимумов
После фильтрации: 21 максимумов и 20 минимумов
Первый максимум: 12.01.2005 = 87.17
Последний максимум: 31.10.2024 = 106.26
Первый минимум: 11.05.2005 = 67.74
Последний минимум: 12.05.2024 = 71.46


In [48]:
df_with_extrems.to_clipboard(index=False, excel=True, sep=";")